# Assignment week 3 part 2
Note the answer to second part is at the end of the notebook, the first part is used to recreate the initial table

## Segmenting and Clustering Neighborhoods in Toronto


### Load required libraries, tools & packages

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# for webscraping import Beautiful Soup 
!pip install BeautifulSoup4
get_ipython().system(u' pip install --upgrade pip')
get_ipython().system(u' pip install beautifulsoup4')
!pip install lxml
!pip install html5lib
!pip install requests

#Importing packages
import requests
from bs4 import BeautifulSoup

import xml

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.2 MB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-1.21.0-py_0

The following packages will be UPDATED:

  openssl                                 1.1.1f-h516909a_0 --> 1.1.1g-h516909a_0



openssl-1.1.1g       | 2.1 MB    | ############

### Web page scraping

In [2]:
url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(url,'lxml')

In [3]:
df_geo = pd.read_csv('http://cocl.us/Geospatial_data')
df_geo.columns = ['PostalCode', 'Latitude', 'Longitude']

### Organise data and creating final data Frame

In [9]:
table_post = soup.find('table')
fields = table_post.find_all('td')

postcode = []
borough = []
neighbourhood = []

for i in range(0, len(fields), 3):
    postcode.append(fields[i].text.strip())
    borough.append(fields[i+1].text.strip())
    neighbourhood.append(fields[i+2].text.strip())
        
df_pc = pd.DataFrame(data=[postcode, borough, neighbourhood]).transpose()
df_pc.columns = ['PostalCode', 'Borough', 'Neighbourhood']

df_pc['Borough'].replace('Not assigned', np.nan, inplace=True)
df_pc.dropna(subset=['Borough'], inplace=True)

df_pc_r = df_pc.groupby(['PostalCode', 'Borough'])['Neighbourhood'].apply(', '.join).reset_index()
df_pc_r.columns = ['PostalCode', 'Borough', 'Neighbourhood']

df_pos = pd.merge(df_pc_r, df_geo, on=['PostalCode'], how='inner')

df_pos_r = df_pos[['PostalCode', 'Borough', 'Neighbourhood', 'Latitude', 'Longitude']].copy()


df_pos_r.head()
df_pos_r.shape

(103, 5)

### Boroughs clustering

In [10]:
# set number of clusters
kclusters = 4

from sklearn.cluster import KMeans
k_means = KMeans(init="k-means++", n_clusters=kclusters, n_init=15)
k_means.fit(df_pos_r[['Latitude', 'Longitude']])

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=4, n_init=15, n_jobs=None, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

### Assign colors to clusters

In [11]:
df_pos_r['Cluster'] = k_means.labels_
df_pos_r['Color'] = 'color'
for i in range(0,df_pos_r.Cluster.count()):
    if df_pos_r.Cluster[i] == 0:
        df_pos_r.loc[i:, 'Color'] = 'blue' 
    elif df_pos_r.Cluster[i] == 1:
        df_pos_r.loc[i:, 'Color'] = 'red'
    elif df_pos_r.Cluster[i] == 2:
        df_pos_r.loc[i:, 'Color'] = 'green'
    else:
        df_pos_r.loc[i:, 'Color'] = 'orange'
df_pos_r.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Cluster,Color
0,M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353,0,blue
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497,0,blue
2,M1E,Scarborough,Guildwood / Morningside / West Hill,43.763573,-79.188711,0,blue
3,M1G,Scarborough,Woburn,43.770992,-79.216917,0,blue
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,0,blue


In [15]:
df_pos_n = df_pos_r[['Cluster','Latitude', 'Longitude', 'Borough', 'Color']].copy()
df_pos_n.head()

,Cluster,Latitude,Longitude,Borough,Color
0,0,43.806686,-79.194353,Scarborough,blue
1,0,43.784535,-79.160497,Scarborough,blue
2,0,43.763573,-79.188711,Scarborough,blue
3,0,43.770992,-79.216917,Scarborough,blue
4,0,43.773136,-79.239476,Scarborough,blue


### get geographical coordinate of Toronto

In [16]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values


address = 'Toronto, CA'

geolocator = Nominatim(user_agent="To_explorer")
location = geolocator.geocode(address)
lat = location.latitude
long = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(lat, long))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [17]:
print(lat)
print(long)
df_pos_r.dtypes

43.6534817
-79.3839347


PostalCode        object
Borough           object
Neighbourhood     object
Latitude         float64
Longitude        float64
Cluster            int32
Color             object
dtype: object

### Cluster mapping

In [20]:
map_clusters = folium.Map(location=[lat, long], zoom_start=12)

# add markers to the map


for Cluster, Lati, Longi, Borough, Color in zip(df_pos_n['Cluster'], df_pos_n['Latitude'], df_pos_n['Longitude'], df_pos_n['Borough'],  df_pos_r['Color']):
    label = folium.Popup(Borough, parse_html=True)
    folium.CircleMarker(
        [Lati, Longi],
        radius=5,
        popup=label,
        color=Color,
        fill=True,
        fill_color= Color,
        fill_opacity=0.7
        
).add_to(map_clusters)  


map_clusters











